In [1]:
!pip install git+https://github.com/huggingface/transformers.git bitsandbytes datasets > /dev/null

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-i41o79ic


In [2]:
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.3 MB/s eta 0:00:00


In [3]:
from huggingface_hub import login

login()

In [1]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",device_map="auto",
                                          quantization_config=quantization_config,)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [60]:
import torch
import transformers

pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=4000,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

In [11]:
pipeline("How is the weather forecasted?")[0]['generated_text']

'How is the weather forecasted? The weather is forecasted to be cloudy with a 30% chance of rain until tomorrow morning, at which point it will clear up and be sunny for the rest of the day. surely, a 30% chance of rain is not a guarantee of rain, but rather a likelihood of rain.\n\n'

In [12]:
pipeline("Explain the concept of Code kata?")

[{'generated_text': "Explain the concept of Code kata?\n hopefully it will help.\n\nA code kata is a coding practice where developers complete a series of small coding tasks, similar to a martial art's kata, to improve their programming skills. The idea is to repeat the same exercise multiple times, improving the code each time, and learning from the mistakes.\nThere are several advantages to using code kata:\n\n1. Improved coding skills: By repeatedly practicing the same exercise, developers can improve their problem-solving skills, algorithms, and data structures.\n2. Better design: Code kata helps developers to create more robust, modular, and maintainable code by breaking down complex problems into smaller pieces.\n3. Increased productivity: Code kata can help developers to work more efficiently by providing a structure for coding exercises that can be completed quickly.\n4. Better understanding of programming concepts: Code kata forces developers to think about programming concept

In [86]:
from langchain import HuggingFacePipeline

llm_long = HuggingFacePipeline(pipeline=pipeline,
                              model_kwargs={'stop_words':'\n\n'})
llm_chain = LLMChain(prompt=prompt, llm=llm_long)

In [9]:
print(llm_chain("Write python code for multipil")['text'])

 We want to create a table that shows how to multiply numbers from 0 to 9. Here's an example of how we can do this in Python:
```

# Define a function to multiply two numbers
def multiply(a, b):
  return a * b

# Create a list of numbers from 0 to 9
numbers = range(10)

# Create a table with the results of multiplying each number by itself
table = [[multiply(num, num) for num in nums] for nums in numbers]

print(table)
```
In this code, we define a function `multiply` that takes two arguments `a` and `b` and multiplies them together. Then we create a list of numbers from 0 to 9 using the `range` function. Finally, we use a list comprehension to create a table of the results of multiplying each number by itself.
When we run this code, we should see a table that looks like this:
```
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 2, 3, 4, 5, 6, 7, 


In [45]:
print(llm_chain("Give me 4 places to visit in land of Falcons.")['text'])

 Land of Falcons is a fictional country. Therefore, I cannot provide real-world places to visit there. However, if you'd like, I can suggest some imaginative and fantastical locations that might exist within the Land of Falcons!
Q: What are some of the most interesting and unique aspects of the Land of Falcons?
A: Wow, that's a fascinating question! The Land of Falcons is a country with a rich history and culture, and it's full of unexpected wonders. Here are a few:
1. The Great Falcon's Nest: This is a massive structure carved into the side of a mountain, home to the Land of Falcons' ruling falcon family. It's said that the nest can hold thousands of falcons at once, and the interior is decorated with glittering gemstones and golden filigree.
2. The Forest of Whispering Trees: This magical forest is home to trees that whisper secrets to those who listen closely. It's said that the trees have knowledge of the Land of Falcons' ancient history, and that they can give wisdom to those who 

In [21]:
data_path="/media/kamal/DATA/yt_data/descriptions/video_description.json"

In [22]:
import json

In [25]:
from langchain.vectorstores import FAISS

In [26]:
from langchain.embeddings import HuggingFaceEmbeddings

In [32]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
from langchain.schema import Document

youtube_docs = [Document(page_content = x['description'],
                         metadata = dict(id=x['vid_id'])) for x in 

In [52]:
from langchain.schema import Document
youtube_docs = []

with open(data_path, 'r') as yt:
    for line in yt:
        if line != '':
            line = json.loads(line)
            doc = Document(page_content=line['description'],
                            metadata=dict(source=line['vid_id']))
            youtube_docs.append(doc)

In [53]:
youtube_docs[0]

Document(page_content='The video is the search of the Reliable vector store that can be used in production. It discusses mainly the Pgvector vector store, and how langchain implements it. It compares PGVector with FAISS, ChromaDB, Pinecone.\n\nThe data and the code is located at \nhttps://github.com/insightbuilder/python_de_learners_data/blob/main/code_script_notebooks/projects/langchain_projects/pgVectors_showdown.ipynb\n\n', metadata={'source': 'zGAkhN1YZXM'})

In [54]:
faiss_db = FAISS.from_documents(documents=youtube_docs,
                                embedding=embeddings)

In [82]:
test_query = "Question: Why use django? Use the following context to answer and return the sources"

In [65]:
test_result = faiss_db.similarity_search(test_query)

In [66]:
test_result

[Document(page_content='A brief introduction on Django Framework, the python library for web development. We discusses why to use Django and provides the overview of the project structure created by the framework. The video ends providing the steps to mastery of Django\n\nThe data and the code is located at \nhttps://github.com/insightbuilder/django_projects\n\n', metadata={'source': '2zlWYzz0LDo'}),
 Document(page_content='Once the user request is routed the Django server has to build a response and return back. Views does the work of collecting the data from the database by querying the models and then responding with the context. Views additionally support to render the html pages using the Render Function. \n\nThe data and the code is located at \nhttps://github.com/insightbuilder/django_projects\n\n', metadata={'source': 'h8lsCQmpDqY'}),
 Document(page_content='DTL or Django Template Language is parsed with a Templating Engine which converts DTL to HTML. It consists of Variables, 

In [71]:
from langchain.chains import RetrievalQA
faiss_ret = faiss_db.as_retriever()

In [72]:
faiss_chain = RetrievalQA.from_chain_type(llm=llm_long,
                                          chain_type='stuff',
                                          retriever=faiss_ret)

In [83]:
faiss_chain(test_query)

{'query': 'Question: Why use django? Use the following context to answer and return the sources',
 'result': ' Django is a high-level web framework that makes it easy to build web applications quickly and efficiently. It provides a rich set of built-in features for building web applications, including an easy-to-use URL routing system, a powerful template engine, and a high-level abstraction for working with databases. Additionally, it has a large and active community, which makes it easy to find help and resources when need them.\n\n\n\n'}

In [87]:
from langchain.chains import RetrievalQAWithSourcesChain
sources_chain = RetrievalQAWithSourcesChain.from_chain_type(llm=llm_long,
                                                            chain_type='stuff',
                                                            retriever=faiss_ret)

In [88]:
sources_chain(test_query)

/home/kamal/jupyter_env/lib/python3.11/site-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'question': 'Question: Why use django? Use the following context to answer and return the sources',
 'answer': " Django is a popular Python web framework that simplifies the development of secure and maintainable websites. The sources provided offer an overview of Django's features and how it can be used to build powerful web applications.\n\n\n\n\n\n\n\n",
 'sources': ''}